In [77]:
#Libraries
import glob
import string
import nltk
import os
from os import listdir
import re
import codecs
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import csv
import math, random
from collections import defaultdict, Counter
from bs4 import BeautifulSoup
import requests
import pandas as pd
from sklearn import preprocessing
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
import statsmodels.api as sm
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB

In [ ]:
#Test cell for coding experiments


In [ ]:
#Data Cleaning
path = 'D:\\uncc2018\\Text\\*.txt'  
files=glob.glob(path)   
for file in files:     
    f = open(file, encoding="ascii", errors="surrogateescape")
    text = f.read()
    print('Cleaning the file:' + os.path.basename(f.name))
    #remove awkward spacing around special characters - there are splits like "de- scribe"
    text = re.sub(r'[-]\s\n', '', text)
    text = re.sub(r'[-]\n', '', text)
    # split into words
    from nltk.tokenize import word_tokenize
    tokens = word_tokenize(text)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuation from each word
    import string
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stop words
    from nltk.corpus import stopwords
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    # stemming of words
    from nltk.stem.porter import PorterStemmer
    porter = PorterStemmer()
    stemmed = [porter.stem(word) for word in words]
    # writing the cleaned data to clean data directory
    cleanedFilePath = "D:\\uncc2018\\cleanedtext\\" + os.path.basename(f.name)
    cleanedFile = open(cleanedFilePath, 'w')
    cleanedFile.writelines(["%s\n" % word  for word in stemmed])
    cleanedFile.close()
print('Clean up completed - Refer to the cleanedText directory for data')

In [ ]:
#Merging files
filenames = [fname for fname in os.listdir('D:\\uncc2018\\cleanedtext\\')] 
with open('D:\\uncc2018\\output\\WordCloudBase.txt','w') as outfile:
    for fname in filenames:
        with open('D:\\uncc2018\\cleanedtext\\' + fname) as infile:
                lines = infile.readlines()
                mystr = ' '.join([line.strip() for line in lines if len(line) > 3])
                outfile.write(mystr + '\n')

In [ ]:
#Executing wordcloud
figure = plt.figure(figsize = (10,10))
with codecs.open('D:\\uncc2018\\output\\WordCloudBase.txt', "r", encoding='utf-8') as abs:
        text = abs.read()
        wc = WordCloud(background_color="white")  
        wc.generate(text)
        plt.imshow(wc, aspect='auto')
        plt.axis("off")
plt.show() 

In [ ]:
#Term-document matrix
from collections import Counter
csvfile = 'D:\\uncc2018\\output\\term_document_matrix.csv'
header_count = 1
f = open(csvfile, 'w+').close()
with codecs.open('D:\\uncc2018\\output\\WordCloudBase.txt', "r", encoding='utf-8') as abs:
    read_abs = abs.read()
    words = read_abs.split()
    Counter = Counter(words)
    most_occur = Counter.most_common(50)
    word_list= [i[0] for i in most_occur]
    header = ['Document Index'] + word_list
    doc = re.split(r'\n', read_abs)
    i = 1
    word_counts = []
    for para in doc:
        int_count = []
        word_count_list =[]
        for k in range(0,(len(word_list))):
            int_count = sum(1 for match in re.finditer(word_list[k], para))
            word_count_list = word_count_list + [int_count]
        word_counts = [i] + word_count_list
        with open(csvfile, 'w', newline='') as myfile:
            wr = csv.writer(myfile)
            if header_count == 1:
                wr.writerow(header)
                header_count = 0
            wr.writerow(word_counts)
        i = i + 1 

In [ ]:
#Reducing the document scope for Topic Modeling
#We ended up not using this because it blurrs the distiction between documents too much
#with open('D:\\uncc2018\\output\\TMBase.txt','w+',newline='') as txt:
    #with codecs.open('D:\\uncc2018\\output\\WordCloudBase.txt', "r", encoding='utf-8') as abs:
        #read_abs = abs.read()
        #remove = '|'.join(word_list)
        #print(remove)
        #out = re.sub(r'\b(?!perform|manag|firm|use|employe|studi|journal|measur|environment|practic|effect|research|resourc|variabl|social|corpor|result|market|product|model|organiz|relationship|busi|industri|may|human|organ|posit|relat|compani|level|work|strategi|turnov|respons|also|strateg|custom|develop|differ|data|csr|report|new|reput|tabl|valu|inform|analysi|control)\b\w+','',read_abs)
        #out = re.sub(' +',' ',out)
        #txt.write(out)

In [ ]:
# Function definitions for Gibbs sampling and Topic modeling
#
def roll_a_die():
    return random.choice([1,2,3,4,5,6])

def direct_sample():
    d1 = roll_a_die()
    d2 = roll_a_die()
    return d1, d1 + d2

def random_y_given_x(x):
    """equally likely to be x + 1, x + 2, ... , x + 6"""
    return x + roll_a_die()

def random_x_given_y(y):
    if y <= 7:
        # if the total is 7 or less, the first die is equally likely to be
        # 1, 2, ..., (total - 1)
        return random.randrange(1, y)
    else:
        # if the total is 7 or more, the first die is equally likely to be
        # (total - 6), (total - 5), ..., 6
        return random.randrange(y - 6, 7)

def gibbs_sample(num_iters=100):
    x, y = 1, 2 # doesn't really matter
    for _ in range(num_iters):
        x = random_x_given_y(y)
        y = random_y_given_x(x)
    return x, y

def compare_distributions(num_samples=1000):
    counts = defaultdict(lambda: [0, 0])
    for _ in range(num_samples):
        counts[gibbs_sample()][0] += 1
        counts[direct_sample()][1] += 1
    return counts

def sample_from(weights):
    total = sum(weights)
    rnd = total * random.random()       # uniform between 0 and total
    for i, w in enumerate(weights):
        rnd -= w                        # return the smallest i such that
        if rnd <= 0: return i           # sum(weights[:(i+1)]) >= rnd

def p_topic_given_document(topic, d, alpha=0.1):
    """the fraction of words in document _d_
    that are assigned to _topic_ (plus some smoothing)"""

    return ((document_topic_counts[d][topic] + alpha) /
            (document_lengths[d] + K * alpha))

def p_word_given_topic(word, topic, beta=0.1):
    """the fraction of words assigned to _topic_
    that equal _word_ (plus some smoothing)"""

    return ((topic_word_counts[topic][word] + beta) /
            (topic_counts[topic] + W * beta))

def topic_weight(d, word, k):
    """given a document and a word in that document,
    return the weight for the k-th topic"""

    return p_word_given_topic(word, k) * p_topic_given_document(k, d)

def choose_new_topic(d, word):
    return sample_from([topic_weight(d, word, k)
                        for k in range(K)])

In [ ]:
#Topic Modeling
# If an error with "Counter not callable" appears, rerun the library import cell
from collections import defaultdict, Counter
abs = codecs.open('D:\\uncc2018\\output\\WordCloudBase.txt', "r", encoding='utf-8')
docs = abs.readlines()
documents = [line.split() for line in docs]

K = 2

document_topic_counts = [Counter() for _ in documents]
topic_word_counts = [Counter() for _ in range(K)]
topic_counts = [0 for _ in range(K)]
document_lengths = [len(d) for d in documents]

distinct_words = set(word for document in documents for word in document)
W = len(distinct_words)
print(distinct_words)
D = len(documents)
print(D)

In [ ]:
#Topic Modeling 2
random.seed(0)
document_topics = [[random.randrange(K) for word in document]
                   for document in documents]

for d in range(D):
    for word, topic in zip(documents[d], document_topics[d]):
        document_topic_counts[d][topic] += 1
        topic_word_counts[topic][word] += 1
        topic_counts[topic] += 1

for iter in range(25):
    for d in range(D):
        for i, (word, topic) in enumerate(zip(documents[d],
                                              document_topics[d])):

            # remove this word / topic from the counts
            # so that it doesn't influence the weights
            document_topic_counts[d][topic] -= 1
            topic_word_counts[topic][word] -= 1
            topic_counts[topic] -= 1
            document_lengths[d] -= 1

            # choose a new topic based on the weights
            new_topic = choose_new_topic(d, word)
            document_topics[d][i] = new_topic

            # and now add it back to the counts
            document_topic_counts[d][new_topic] += 1
            topic_word_counts[new_topic][word] += 1
            topic_counts[new_topic] += 1
            document_lengths[d] += 1

In [ ]:
# Topic Modeling 3
csvfile2 = 'D:\\uncc2018\\output\\TMK2_full.csv'
for k, word_counts in enumerate(topic_word_counts):
    for word, count in word_counts.most_common():
        with open(csvfile2, 'a+', newline='') as myfile2:
            wr = csv.writer(myfile2)
            if count > 800: 
                wr.writerow([k, word, count])

In [ ]:
#Topic modeling 4 - final step
topic_names = ["Result-oriented","Socially Responsible"]

csvfile3 = 'D:\\uncc2018\\output\\TMwDocs.csv'
for document, topic_counts in zip(documents, document_topic_counts):
    with open(csvfile3, 'a+', newline='') as myfile3:
        wr = csv.writer(myfile3)
        for topic, count in topic_counts.most_common():
            if count > 0:
                wr.writerow([topic_names[topic], count])
        wr.writerow('')

In [81]:
#Modeling - Logistic regression and Bayes
#If you get "numpy object not callable" error, rerun the libraries cell
from sklearn.metrics import confusion_matrix
regdata = pd.read_csv('D:\\uncc2018\\output\\modelinput.csv')
target=['class']
predict=['perform','manag','firm','use','employe','studi','journal','measur','environment','practic','effect','research','resourc','variabl','social','corpor','result','market','product','model','organiz','relationship','busi','industri','may','human','organ','posit','relat','compani','level','work','strategi','turnov','respons','also','strateg','custom','develop','differ','data','csr','report','new','reput','tabl','valu','inform','analysi', 'control']
x=regdata[predict]
y=regdata[target]
logreg = LogisticRegression(solver='lbfgs', max_iter=500)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=0)
logreg.fit(x_train, y_train.values.ravel())
y_pred = logreg.predict(x_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(x_test, y_test)))
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)
print()
from sklearn.metrics import confusion_matrix
model = MultinomialNB().fit(x_train, y_train.values.ravel())
predicted = model.predict(x_test)
print("Gaussian Naive Bayes model accuracy:", metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, predicted))

Accuracy of logistic regression classifier on test set: 0.84
[[29  6]
 [ 4 24]]

Gaussian Naive Bayes model accuracy: 0.8412698412698413
[[33  2]
 [ 3 25]]
